# Tutorial HANNAH Framework 
## AutoML School
### Hannover 2024 

### Create Search Space
First we create a simple search space. This search space description is written to a file such that the `hydra` config framework
can refer to it and instantiate the search space. 
> Note: Usually, search spaces are located in `hannah/models`.

The search space consists of a variable number of `conv_relu` blocks with a `linear` layer at the end. 



In [1]:
%%writefile space.py
from hannah.nas.functional_operators.op import ChoiceOp, Tensor, scope
from hannah.nas.functional_operators.operators import AdaptiveAvgPooling, Conv2d, Linear, Relu
from hannah.nas.functional_operators.op import search_space
from hannah.nas.parameters.parameters import CategoricalParameter, IntScalarParameter


@scope
def conv_relu(input, kernel_size, out_channels, stride):
    in_channels = input.shape()[1]
    weight = Tensor(name='weight',
                    shape=(out_channels, in_channels, kernel_size, kernel_size),
                    axis=('O', 'I', 'kH', 'kW'),
                    grad=True)

    conv = Conv2d(stride=stride)(input, weight)
    relu = Relu()(conv)
    return relu


def adaptive_avg_pooling(input):
    return AdaptiveAvgPooling()(input)


def dynamic_depth(*exits, switch):
    return ChoiceOp(*exits, switch=switch)()


def linear(input, out_features):
    input_shape = input.shape()
    in_features = input_shape[1] * input_shape[2] * input_shape[3]
    weight = Tensor(name='weight',
                    shape=(in_features, out_features),
                    axis=('in_features', 'out_features'),
                    grad=True)

    out = Linear()(input, weight)
    return out


@scope
def classifier_head(input, num_classes):
    out = adaptive_avg_pooling(input)
    out = linear(out, num_classes)
    return out


@search_space
def cnn_search_space(name, input, max_blocks=3, max_channels=64, num_classes=10):
    out_channels = IntScalarParameter(16, max_channels, step_size=8, name="out_channels")
    kernel_size = CategoricalParameter([3, 5, 7, 9], name="kernel_size")
    stride = CategoricalParameter([1, 2], name="stride")
    num_blocks = IntScalarParameter(1, max_blocks, name="num_blocks")
    exits = []
    out = input
    for i in range(num_blocks.max + 1):
        out = conv_relu(out,
                        stride=stride.new(),
                        out_channels=out_channels.new(),
                        kernel_size=kernel_size.new())
        exits.append(out)

    out = dynamic_depth(*exits, switch=num_blocks)
    out = classifier_head(out, num_classes=num_classes)
    return out

Overwriting space.py


Now we create a config file for `hydra`:

In [2]:
from omegaconf import OmegaConf
import os

cfg_dict ={"_target_": "space.cnn_search_space",
           "name": "cnn_search_space",
           "num_classes": 10,
           "max_channels": 64}

cfg = OmegaConf.create(cfg_dict)

if not os.path.exists("model"):
    os.mkdir("model")
OmegaConf.save(config=cfg, f="model/cnn_search_space.yaml")


### Define Hardware Description

### Configure NAS

### Run NAS

Now we can run the NAS in the "terminal". Note, that we have to append the current working directory to the PYTHONPATH such 
that the search space located in our current directory can be found. If the model file where to be located in the `hannah/models/`
directory that would not be necessary.

In [1]:
%%bash
export PYTHONPATH=$PYTHONPATH:.
hannah-train nas=random_nas model=cnn_search_space nas.n_jobs=1


 **                                            **
/**                                           /**
/**       ******   *******  *******   ******  /**
/******  //////** //**///**//**///** //////** /******
/**///**  *******  /**  /** /**  /**  ******* /**///**
/**  /** **////**  /**  /** /**  /** **////** /**  /**
/**  /**//******** ***  /** ***  /**//********/**  /**
//   //  //////// ///   // ///   //  //////// //   //

[2024-08-29 16:40:21,845][hannah.utils.utils][INFO] - Environment info:
[2024-08-29 16:40:23,521][hannah.utils.utils][INFO] -   Number of GPUs: 1
[2024-08-29 16:40:23,521][hannah.utils.utils][INFO] -   CUDA version: 12.1
[2024-08-29 16:40:23,529][hannah.utils.utils][INFO] -   CUDNN version: 8902
[2024-08-29 16:40:23,529][hannah.utils.utils][INFO] -   Kernel: 5.15.153.1-microsoft-standard-WSL2
[2024-08-29 16:40:23,529][hannah.utils.utils][INFO] -   Python: 3.10.12 (main, Jul 29 2024, 16:56:48) [GCC 11.4.0]
[2024-08-29 16:40:23,529][hannah.utils.utils][INFO] -   PyTorch:

INFO: Seed set to 1234


[2024-08-29 16:41:29,024][lightning.fabric.utilities.seed][INFO] - Seed set to 1234
[2024-08-29 16:41:29,030][hannah.callbacks.optimization][INFO] - Monitoring the following values for optimization
[2024-08-29 16:41:29,031][hannah.callbacks.optimization][INFO] -   - val_error direction: minimize(1)
[2024-08-29 16:41:29,152][pytorch_lightning.utilities.rank_zero][INFO] - GPU available: True (cuda), used: True
[2024-08-29 16:41:29,152][pytorch_lightning.utilities.rank_zero][INFO] - TPU available: False, using: 0 TPU cores
[2024-08-29 16:41:29,152][pytorch_lightning.utilities.rank_zero][INFO] - IPU available: False, using: 0 IPUs
[2024-08-29 16:41:29,152][pytorch_lightning.utilities.rank_zero][INFO] - HPU available: False, using: 0 HPUs
[2024-08-29 16:41:29,153][pytorch_lightning.utilities.rank_zero][INFO] - `Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
[2024-08-29 16:41:29,153][pytorch_lightning.utilities.rank_zero][INFO] - `Trainer(lim

Epoch 7: 100%|██████████| 421/421 [00:15<00:00, 26.58it/s, v_num=0, train_loss=0.0147, val_loss=0.0452]  
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 8: 100%|██████████| 421/421 [00:15<00:00, 26.97it/s, v_num=0, train_loss=0.000486, val_loss=0.0406]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 8: 100%|██████████| 421/421 [00:17<00:00, 24.75it/s, v_num=0, train_loss=0.000486, val_loss=0.0396][2024-08-29 16:44:07,956][pytorch_lightning.utilities.rank_zero][INFO] - Epoch 8, global step 3789: 'val_error' reached 0.00985 (best 0.00985), saving model to '/home/moritz/projects/hannah/tutorials/automlschool24/0/checkpoints/epoch=8-step=3789.ckpt' as top 1


### Evaluate